In [113]:
from pathlib import Path
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import Error
from datetime import datetime

# project = mysql.connector.connect(
#     host="localhost",
#     user="root",
#     password="Ff301128"
#     )

# mycursor = project.cursor()

#----------------------------------------------------------------------------------------------------------------------------
#use concat to merge all of files together
list_stacked = pd.DataFrame() 
current_path = Path()/'CSV'
for folder in current_path.absolute().glob('*'):
    for file in folder.glob('H_lvr_land_A.csv'):
        data = pd.read_csv(file)[1:]
        list_stacked = pd.concat([list_stacked, data])

#----------------------------------------------------------------------------------------------------------------------------
#Originally, there are 33 columns in the file; select 18 columns which need to be analyzed
data = list_stacked[['鄉鎮市區', '土地位置建物門牌', '土地移轉總面積平方公尺',
                '交易年月日', '總樓層數', '建物型態',
                '建物移轉總面積平方公尺', '建物現況格局-房', '建物現況格局-廳',
                '建物現況格局-衛', '有無管理組織', '總價元',
                '單價元平方公尺', '車位類別', '車位總價元', 
                '主建物面積', '陽台面積', '電梯']]

#----------------------------------------------------------------------------------------------------------------------------
#change the object type of columns into float type of columns
for column in pd.DataFrame(data, columns = ['土地移轉總面積平方公尺','建物移轉總面積平方公尺', '建物現況格局-房', '建物現況格局-廳',
                '建物現況格局-衛', '總價元', '單價元平方公尺', '車位總價元', '主建物面積', '陽台面積',]):
    data = data.copy()
    data[column] = data[column].astype(float)

#----------------------------------------------------------------------------------------------------------------------------
#坪 is the most popular unit that Taiwanese to measure the property, so we need to transfer from meter squared to 坪
data['土地移轉總面積_坪'] = data['土地移轉總面積平方公尺']*0.3025
data['建物移轉總面積_坪'] = data['建物移轉總面積平方公尺']*0.3025
data['單價元_坪'] = data['單價元平方公尺']*0.3025
data['主建物面積_坪'] = data['主建物面積']*0.3025
data['陽台面積_坪'] = data['陽台面積']*0.3025

#----------------------------------------------------------------------------------------------------------------------------
#change date of purchase from object type to int and create year, quarter, month and date
data = data.copy()
data['交易年月日'] = pd.to_datetime((data['交易年月日'].astype(int)+19110000).astype(str), format = '%Y%m%d', errors='coerce')
data['Year'] = data['交易年月日'].dt.year.fillna(0).astype(int)
data['Quarter'] = data['交易年月日'].dt.quarter.fillna(0).astype(int)
data['Month'] = data['交易年月日'].dt.month.fillna(0).astype(int)
data['Day'] = data['交易年月日'].dt.day.fillna(0).astype(int)


data


,鄉鎮市區,土地位置建物門牌,土地移轉總面積平方公尺,交易年月日,總樓層數,建物型態,建物移轉總面積平方公尺,建物現況格局-房,建物現況格局-廳,建物現況格局-衛,...,電梯,土地移轉總面積_坪,建物移轉總面積_坪,單價元_坪,主建物面積_坪,陽台面積_坪,Year,Quarter,Month,Day
1,中壢區,三座屋段舊社小段172-1地號,241.11,2020-12-01,NaN,其他,0.00,0.0,0.0,0.0,...,無,72.935775,0.000000,23333.3375,0.00000,0.000000,2020,4,12,1
2,大溪區,石園路７７６巷２５０弄８０號,96.72,2020-12-01,一層,公寓(5樓含以下無電梯),19.92,3.0,2.0,1.0,...,無,29.257800,6.025800,15185.8025,6.02580,0.000000,2020,4,12,1
3,桃園區,文中路７４９巷４號五樓,18.12,2020-12-03,八層,華廈(10層含以下有電梯),99.74,3.0,2.0,2.0,...,有,5.481300,30.171350,15012.7725,23.52240,3.003825,2020,4,12,3
4,中壢區,自立段675地號,489.00,2020-12-01,NaN,其他,0.00,0.0,0.0,0.0,...,無,147.922500,0.000000,5864.5675,0.00000,0.000000,2020,4,12,1
5,桃園區,樹林八街３０之１號八樓,17.38,2020-12-01,十二層,住宅大樓(11層含以上有電梯),145.89,2.0,2.0,1.0,...,有,5.257450,44.131725,18039.2850,17.16385,2.026750,2020,4,12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12530,中壢區,中北段152地號,138.67,2009-08-11,NaN,其他,0.00,0.0,0.0,0.0,...,無,41.947675,0.000000,4575.0100,0.00000,0.000000,2009,3,8,11
12531,中壢區,桃園市中壢區中華路一段７６３號,109.86,2008-03-22,四層,透天厝,285.43,3.0,2.0,2.0,...,無,33.232650,86.342575,15685.2300,69.98035,4.806725,2008,1,3,22
12532,龜山區,舊路坑一段178地號,161.39,2004-02-05,NaN,其他,0.00,0.0,0.0,0.0,...,無,48.820475,0.000000,3373.7825,0.00000,0.000000,2004,1,2,5
12533,龜山區,舊路坑一段178地號,162.03,2004-02-14,NaN,其他,0.00,0.0,0.0,0.0,...,無,49.014075,0.000000,3659.3425,0.00000,0.000000,2004,1,2,14
